# Ablation study with oracle

In [1]:
import sys
import os
sys.path.append('../')

import pandas as pd

from tsGOMP import tsGOMP_OneAssociation
from associations import PearsonMultivariate, SpearmanMultivariate
from models import VARModel, ARDLModel
import data_opener

import numpy as np
import networkx as nx
from cdt.metrics import SHD
import matplotlib.pyplot as plt
import time


random_generator = np.random.default_rng(0)

/home/etienne/.local/lib/python3.8/site-packages/mass_ts/_mass_ts.py:17: UserWarning: GPU support will not work. You must pip install mass-ts[gpu].
  warnings.warn(
No GPU automatically detected. Setting SETTINGS.GPU to 0, and SETTINGS.NJOBS to cpu_count.


In [2]:
def separate_train_test(df):
    df_train = df.iloc[:int(0.7*len(df))]
    df_test = df.iloc[int(0.7*len(df)):]
    return df_train, df_test

In [8]:
def run_trials(rootdir,dataset, config,
               select_random_targets = False, number_selections = 100,
               seed=0):
    
    rng = np.random.default_rng(seed)
    
    df_results = []
    
    for filename in os.listdir(rootdir + "data/" + dataset + "/"):
        if not os.path.isfile(rootdir + "data/" + dataset + "/" + filename):
            continue
        print("File",filename,"has begun.")

        df, _, causes = data_opener.open_dataset_and_ground_truth(dataset, filename, rootdir=rootdir)
        
        df_train, df_test = separate_train_test(df)

        targets = rng.choice(df.columns, size=number_selections) if select_random_targets else df.columns

        for target in targets:
            if len(causes[target]) == 0:
                continue
            else:
                print("\tVariable", target, "is beginning.")
                
                ground_truth = causes[target].copy()

                time_algostart = time.time()
                algo = tsGOMP_OneAssociation(config, target, ground_truth, verbosity=1)
                algo.fit(df_train)
                time_algoend = time.time()


                lPred = algo.get_selected_features()
                sPred = set(lPred)
                sTrue = set(causes[target])

                # metrics from results
                sTP = sTrue & sPred
                recall = len(sTP)/len(sTrue)
                precision = len(sTP)/len(sPred) if len(sPred)>0 else None

                # metrics from model made with only causes
                causes_autoregressive = causes[target] if target in causes[target] else causes[target]+[target]
                model = config["model"](config["model.config"], target)
                # potentially remove the target from the predictors with lag=0, but for now let this be.
                model.fit(df_train[causes_autoregressive])
                train_statistics = model.statistics()
                train_statistics = dict([("causal_train_"+x,train_statistics[x]) for x in train_statistics])
                try:
                    test_statistics = model.statistics(df_test[causes_autoregressive])
                except ValueError:
                    test_statistics = dict()
                test_statistics = dict([("causal_test_"+x,test_statistics[x]) for x in test_statistics])
                
                # compute final test performance
                

                # base record row
                base_row = {"dataset":dataset,"filename":filename,'target':target,
                            "totaltime":time_algoend - time_algostart,
                            "final_precision":precision,"final_recall":recall,
                            "final_TP":len(sTP), 
                            "final_FP":len(sPred)-len(sTP),
                            "final_FN": len(sTrue)-len(sTP), 
                            "final_TN": len(df.columns)+len(sTP)-len(sPred)-len(sTrue)}

                for iteration in algo.history:
                    model = iteration["model"]
                    new_row = {**base_row, "step":iteration["step"],
                               "association_chosen":iteration["association_chosen"],
                               "chosen_in_ground_truth":iteration["chosen_in_ground_truth"],
                               "stopping_metric":iteration["stopping_metric"],
                               "previous_model_time":iteration["previous_model_time"],
                               "associations_time":iteration["associations_time"],
                               "associations_TP": iteration["associations_TP"],
                               "associations_FP": iteration["associations_FP"],
                               "associations_TN": iteration["associations_TN"],
                               "associations_FN": iteration["associations_FN"],
                               "remaining_causal":iteration["remaining_causal"],
                               "current_is_last_model":iteration["current_is_last_model"],
                               "should_have_stopped": iteration["should_have_stopped"]}
                    
                    # model descriptors
                    train_stat = model.statistics()
                    train_stat = dict([("train_"+x,train_stat[x]) for x in train_stat])
                    # the columns can be accessed by model.data.columns
                    test_stat = model.statistics(df_test[model.data.columns])
                    test_stat = dict([("test_"+x,test_stat[x]) for x in test_stat])
                    
                    if iteration["current_is_last_model"]:
                        new_row = {**new_row,**train_stat, **test_stat, **train_statistics, **test_statistics}
                    else:
                        new_row = {**new_row,**train_stat, **test_stat}

                    df_results.append(new_row)
    return df_results

# Automatize

## 7ts2h

### 7ts2h - ARDL - Pearson - F-test

In [4]:
rootdir = "../"
dataset = "SynthNonlin/7ts2h"


association_list = PearsonMultivariate
associationName = "Pearson"
association_config_list = {"return_type": "p-value",
                           "lags": 10,
                           "selection_rule": "max"}


model_constructor = ARDLModel
modelName = "ARDL"
model_config = {
    "constructor":{
            "lags": 10,
            "order": 10,
            "causal": True,
            "trend": "ct",
            "seasonal": False,
            "period": None,
            "missing": "drop"
        },
        "fit":{
            "cov_type": "HC0"
        }
    }

stoppingName = "f-test"
config = {"significance_threshold": 0.05,
          "method": "f-test",
          "max_features": 7,
          "choose_oracle": True,
          "stop_oracle": False,
          "association": association_list,
          "association.config": association_config_list,
          "model": model_constructor,
          "model.config": model_config}

df_results = run_trials(rootdir,dataset, config)

df = pd.DataFrame(df_results)
df["model"]=modelName
df["association"]=associationName
df["stopping_criterion"]=stoppingName
df.to_csv("ARDL_7ts2h.csv",index=False)

File data_1.csv has begun.
	Variable E is beginning.
	Variable B is beginning.
	Variable F is beginning.
	Variable C is beginning.
	Variable H is beginning.
	Variable D is beginning.
	Variable A is beginning.
File data_0.csv has begun.
	Variable E is beginning.
	Variable B is beginning.
	Variable F is beginning.
	Variable C is beginning.
	Variable H is beginning.
	Variable D is beginning.
	Variable A is beginning.
File data_2.csv has begun.
	Variable E is beginning.
	Variable B is beginning.
	Variable F is beginning.
	Variable C is beginning.
	Variable H is beginning.
	Variable D is beginning.
	Variable A is beginning.
File data_6.csv has begun.
	Variable E is beginning.
	Variable B is beginning.
	Variable F is beginning.
	Variable C is beginning.
	Variable H is beginning.
	Variable D is beginning.
	Variable A is beginning.
File data_9.csv has begun.
	Variable E is beginning.
	Variable B is beginning.
	Variable F is beginning.
	Variable C is beginning.
	Variable H is beginning.
	Variab

### 7ts2h - ARLD - Spearman - F-test

In [5]:
rootdir = "../"
dataset = "SynthNonlin/7ts2h"


association_list = SpearmanMultivariate
associationName = "Spearman"
association_config_list = {"return_type": "p-value",
                           "lags": 10,
                           "selection_rule": "max"}


model_constructor = ARDLModel
modelName = "ARDL"
model_config = {
    "constructor":{
            "lags": 10,
            "order": 10,
            "causal": True,
            "trend": "ct",
            "seasonal": False,
            "period": None,
            "missing": "drop"
        },
        "fit":{
            "cov_type": "HC0"
        }
    }

stoppingName = "f-test"
config = {"significance_threshold": 0.05,
          "method": "f-test",
          "max_features": 7,
          "choose_oracle": True,
          "stop_oracle": False,
          "association": association_list,
          "association.config": association_config_list,
          "model": model_constructor,
          "model.config": model_config}

df_results = run_trials(rootdir,dataset, config)

df = pd.DataFrame(df_results)
df["model"]=modelName
df["association"]=associationName
df["stopping_criterion"]=stoppingName
df.to_csv("ARDL_7ts2h.csv", mode="a", index=False, header=False)

File data_1.csv has begun.
	Variable E is beginning.
	Variable B is beginning.
	Variable F is beginning.
	Variable C is beginning.
	Variable H is beginning.
	Variable D is beginning.
	Variable A is beginning.
File data_0.csv has begun.
	Variable E is beginning.
	Variable B is beginning.
	Variable F is beginning.
	Variable C is beginning.
	Variable H is beginning.
	Variable D is beginning.
	Variable A is beginning.
File data_2.csv has begun.
	Variable E is beginning.
	Variable B is beginning.
	Variable F is beginning.
	Variable C is beginning.
	Variable H is beginning.
	Variable D is beginning.
	Variable A is beginning.
File data_6.csv has begun.
	Variable E is beginning.
	Variable B is beginning.
	Variable F is beginning.
	Variable C is beginning.
	Variable H is beginning.
	Variable D is beginning.
	Variable A is beginning.
File data_9.csv has begun.
	Variable E is beginning.
	Variable B is beginning.
	Variable F is beginning.
	Variable C is beginning.
	Variable H is beginning.
	Variab

### 7ts2h - ARLD - Pearson - Wald

In [6]:
rootdir = "../"
dataset = "SynthNonlin/7ts2h"


association_list = PearsonMultivariate
associationName = "Pearson"
association_config_list = {"return_type": "p-value",
                           "lags": 10,
                           "selection_rule": "max"}


model_constructor = ARDLModel
modelName = "ARDL"
model_config = {
    "constructor":{
            "lags": 10,
            "order": 10,
            "causal": True,
            "trend": "ct",
            "seasonal": False,
            "period": None,
            "missing": "drop"
        },
        "fit":{
            "cov_type": "HC0"
        }
    }

stoppingName = "wald-test"
config = {"significance_threshold": 0.05,
          "method": "wald-test",
          "max_features": 7,
          "choose_oracle": True,
          "stop_oracle": False,
          "association": association_list,
          "association.config": association_config_list,
          "model": model_constructor,
          "model.config": model_config}

df_results = run_trials(rootdir,dataset, config)

df = pd.DataFrame(df_results)
df["model"]=modelName
df["association"]=associationName
df["stopping_criterion"]=stoppingName
df.to_csv("ARDL_7ts2h.csv", mode="a", index=False, header=False)

File data_1.csv has begun.
	Variable E is beginning.
	Variable B is beginning.
	Variable F is beginning.
	Variable C is beginning.
	Variable H is beginning.
	Variable D is beginning.
	Variable A is beginning.
File data_0.csv has begun.
	Variable E is beginning.
	Variable B is beginning.
	Variable F is beginning.
	Variable C is beginning.
	Variable H is beginning.
	Variable D is beginning.
	Variable A is beginning.
File data_2.csv has begun.
	Variable E is beginning.
	Variable B is beginning.
	Variable F is beginning.
	Variable C is beginning.
	Variable H is beginning.
	Variable D is beginning.
	Variable A is beginning.
File data_6.csv has begun.
	Variable E is beginning.
	Variable B is beginning.
	Variable F is beginning.
	Variable C is beginning.
	Variable H is beginning.
	Variable D is beginning.
	Variable A is beginning.
File data_9.csv has begun.
	Variable E is beginning.
	Variable B is beginning.
	Variable F is beginning.
	Variable C is beginning.
	Variable H is beginning.
	Variab

### 7ts2h - ARLD - Spearman - Wald

In [7]:
rootdir = "../"
dataset = "SynthNonlin/7ts2h"


association_list = SpearmanMultivariate
associationName = "Spearman"
association_config_list = {"return_type": "p-value",
                           "lags": 10,
                           "selection_rule": "max"}


model_constructor = ARDLModel
modelName = "ARDL"
model_config = {
    "constructor":{
            "lags": 10,
            "order": 10,
            "causal": True,
            "trend": "ct",
            "seasonal": False,
            "period": None,
            "missing": "drop"
        },
        "fit":{
            "cov_type": "HC0"
        }
    }

stoppingName = "wald-test"
config = {"significance_threshold": 0.05,
          "method": "wald-test",
          "max_features": 7,
          "choose_oracle": True,
          "stop_oracle": False,
          "association": association_list,
          "association.config": association_config_list,
          "model": model_constructor,
          "model.config": model_config}

df_results = run_trials(rootdir,dataset, config)

df = pd.DataFrame(df_results)
df["model"]=modelName
df["association"]=associationName
df["stopping_criterion"]=stoppingName
df.to_csv("ARDL_7ts2h.csv", mode="a", index=False, header=False)

File data_1.csv has begun.
	Variable E is beginning.
	Variable B is beginning.
	Variable F is beginning.
	Variable C is beginning.
	Variable H is beginning.
	Variable D is beginning.
	Variable A is beginning.
File data_0.csv has begun.
	Variable E is beginning.
	Variable B is beginning.
	Variable F is beginning.
	Variable C is beginning.
	Variable H is beginning.
	Variable D is beginning.
	Variable A is beginning.
File data_2.csv has begun.
	Variable E is beginning.
	Variable B is beginning.
	Variable F is beginning.
	Variable C is beginning.
	Variable H is beginning.
	Variable D is beginning.
	Variable A is beginning.
File data_6.csv has begun.
	Variable E is beginning.
	Variable B is beginning.
	Variable F is beginning.
	Variable C is beginning.
	Variable H is beginning.
	Variable D is beginning.
	Variable A is beginning.
File data_9.csv has begun.
	Variable E is beginning.
	Variable B is beginning.
	Variable F is beginning.
	Variable C is beginning.
	Variable H is beginning.
	Variab

### 7ts2h - ARLD - Pearson - LR-test

In [8]:
rootdir = "../"
dataset = "SynthNonlin/7ts2h"


association_list = PearsonMultivariate
associationName = "Pearson"
association_config_list = {"return_type": "p-value",
                           "lags": 10,
                           "selection_rule": "max"}


model_constructor = ARDLModel
modelName = "ARDL"
model_config = {
    "constructor":{
            "lags": 10,
            "order": 10,
            "causal": True,
            "trend": "ct",
            "seasonal": False,
            "period": None,
            "missing": "drop"
        },
        "fit":{
            "cov_type": "HC0"
        }
    }

stoppingName = "lr-test"
config = {"significance_threshold": 0.05,
          "method": "lr-test",
          "max_features": 7,
          "choose_oracle": True,
          "stop_oracle": False,
          "association": association_list,
          "association.config": association_config_list,
          "model": model_constructor,
          "model.config": model_config}

df_results = run_trials(rootdir,dataset, config)

df = pd.DataFrame(df_results)
df["model"]=modelName
df["association"]=associationName
df["stopping_criterion"]=stoppingName
df.to_csv("ARDL_7ts2h.csv", mode="a", index=False, header=False)

File data_1.csv has begun.
	Variable E is beginning.
	Variable B is beginning.
	Variable F is beginning.
	Variable C is beginning.
	Variable H is beginning.
	Variable D is beginning.
	Variable A is beginning.
File data_0.csv has begun.
	Variable E is beginning.
	Variable B is beginning.
	Variable F is beginning.
	Variable C is beginning.
	Variable H is beginning.
	Variable D is beginning.
	Variable A is beginning.
File data_2.csv has begun.
	Variable E is beginning.
	Variable B is beginning.
	Variable F is beginning.
	Variable C is beginning.
	Variable H is beginning.
	Variable D is beginning.
	Variable A is beginning.
File data_6.csv has begun.
	Variable E is beginning.
	Variable B is beginning.
	Variable F is beginning.
	Variable C is beginning.
	Variable H is beginning.
	Variable D is beginning.
	Variable A is beginning.
File data_9.csv has begun.
	Variable E is beginning.
	Variable B is beginning.
	Variable F is beginning.
	Variable C is beginning.
	Variable H is beginning.
	Variab

### 7ts2h - ARLD - Spearman - LR-test

In [9]:
rootdir = "../"
dataset = "SynthNonlin/7ts2h"


association_list = SpearmanMultivariate
associationName = "Spearman"
association_config_list = {"return_type": "p-value",
                           "lags": 10,
                           "selection_rule": "max"}


model_constructor = ARDLModel
modelName = "ARDL"
model_config = {
    "constructor":{
            "lags": 10,
            "order": 10,
            "causal": True,
            "trend": "ct",
            "seasonal": False,
            "period": None,
            "missing": "drop"
        },
        "fit":{
            "cov_type": "HC0"
        }
    }

stoppingName = "lr-test"
config = {"significance_threshold": 0.05,
          "method": "lr-test",
          "max_features": 7,
          "choose_oracle": True,
          "stop_oracle": False,
          "association": association_list,
          "association.config": association_config_list,
          "model": model_constructor,
          "model.config": model_config}

df_results = run_trials(rootdir,dataset, config)

df = pd.DataFrame(df_results)
df["model"]=modelName
df["association"]=associationName
df["stopping_criterion"]=stoppingName
df.to_csv("ARDL_7ts2h.csv", mode="a", index=False, header=False)

File data_1.csv has begun.
	Variable E is beginning.
	Variable B is beginning.
	Variable F is beginning.
	Variable C is beginning.
	Variable H is beginning.
	Variable D is beginning.
	Variable A is beginning.
File data_0.csv has begun.
	Variable E is beginning.
	Variable B is beginning.
	Variable F is beginning.
	Variable C is beginning.
	Variable H is beginning.
	Variable D is beginning.
	Variable A is beginning.
File data_2.csv has begun.
	Variable E is beginning.
	Variable B is beginning.
	Variable F is beginning.
	Variable C is beginning.
	Variable H is beginning.
	Variable D is beginning.
	Variable A is beginning.
File data_6.csv has begun.
	Variable E is beginning.
	Variable B is beginning.
	Variable F is beginning.
	Variable C is beginning.
	Variable H is beginning.
	Variable D is beginning.
	Variable A is beginning.
File data_9.csv has begun.
	Variable E is beginning.
	Variable B is beginning.
	Variable F is beginning.
	Variable C is beginning.
	Variable H is beginning.
	Variab

## FMRI

### fMRI - ARDL - Pearson - F-test

In [10]:
rootdir = "../"
dataset = "fMRI_processed_by_Nauta/returns/our_selection"


association_list = PearsonMultivariate
associationName = "Pearson"
association_config_list = {"return_type": "p-value",
                           "lags": 5,
                           "selection_rule": "max"}


model_constructor = ARDLModel
modelName = "ARDL"
model_config = {
    "constructor":{
            "lags": 5,
            "order": 5,
            "causal": True,
            "trend": "ct",
            "seasonal": False,
            "period": None,
            "missing": "drop"
        },
        "fit":{
            "cov_type": "HC0"
        }
    }

stoppingName = "f-test"
config = {"significance_threshold": 0.05,
          "method": "f-test",
          "max_features": 5,
          "choose_oracle": True,
          "stop_oracle": False,
          "association": association_list,
          "association.config": association_config_list,
          "model": model_constructor,
          "model.config": model_config}

df_results = run_trials(rootdir,dataset, config)

df = pd.DataFrame(df_results)
df["model"]=modelName
df["association"]=associationName
df["stopping_criterion"]=stoppingName
df.to_csv("ARDL_fmri.csv",index=False)

File timeseries21.csv has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File timeseries7.csv has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File timeseries23.csv has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File timeseries15.csv has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File timeseries16.csv has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File timeseries18.csv has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File timeseries8.csv ha

### fMRI - ARDL - Spearman - F-test

In [11]:
rootdir = "../"
dataset = "fMRI_processed_by_Nauta/returns/our_selection"


association_list = SpearmanMultivariate
associationName = "Spearman"
association_config_list = {"return_type": "p-value",
                           "lags": 5,
                           "selection_rule": "max"}


model_constructor = ARDLModel
modelName = "ARDL"
model_config = {
    "constructor":{
            "lags": 5,
            "order": 5,
            "causal": True,
            "trend": "ct",
            "seasonal": False,
            "period": None,
            "missing": "drop"
        },
        "fit":{
            "cov_type": "HC0"
        }
    }

stoppingName = "f-test"
config = {"significance_threshold": 0.05,
          "method": "f-test",
          "max_features": 5,
          "choose_oracle": True,
          "stop_oracle": False,
          "association": association_list,
          "association.config": association_config_list,
          "model": model_constructor,
          "model.config": model_config}

df_results = run_trials(rootdir,dataset, config)

df = pd.DataFrame(df_results)
df["model"]=modelName
df["association"]=associationName
df["stopping_criterion"]=stoppingName
df.to_csv("ARDL_fmri.csv", mode="a", index=False, header=False)

File timeseries21.csv has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File timeseries7.csv has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File timeseries23.csv has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File timeseries15.csv has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File timeseries16.csv has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File timeseries18.csv has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File timeseries8.csv ha

### fMRI - ARDL - Pearson - wald-test

In [12]:
rootdir = "../"
dataset = "fMRI_processed_by_Nauta/returns/our_selection"


association_list = PearsonMultivariate
associationName = "Pearson"
association_config_list = {"return_type": "p-value",
                           "lags": 5,
                           "selection_rule": "max"}


model_constructor = ARDLModel
modelName = "ARDL"
model_config = {
    "constructor":{
            "lags": 5,
            "order": 5,
            "causal": True,
            "trend": "ct",
            "seasonal": False,
            "period": None,
            "missing": "drop"
        },
        "fit":{
            "cov_type": "HC0"
        }
    }

stoppingName = "wald-test"
config = {"significance_threshold": 0.05,
          "method": "wald-test",
          "max_features": 5,
          "choose_oracle": True,
          "stop_oracle": False,
          "association": association_list,
          "association.config": association_config_list,
          "model": model_constructor,
          "model.config": model_config}

df_results = run_trials(rootdir,dataset, config)

df = pd.DataFrame(df_results)
df["model"]=modelName
df["association"]=associationName
df["stopping_criterion"]=stoppingName
df.to_csv("ARDL_fmri.csv", mode="a", index=False, header=False)

File timeseries21.csv has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File timeseries7.csv has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File timeseries23.csv has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File timeseries15.csv has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File timeseries16.csv has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File timeseries18.csv has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File timeseries8.csv ha

### fMRI - ARDL - Spearman - wald-test

In [13]:
rootdir = "../"
dataset = "fMRI_processed_by_Nauta/returns/our_selection"


association_list = SpearmanMultivariate
associationName = "Spearman"
association_config_list = {"return_type": "p-value",
                           "lags": 5,
                           "selection_rule": "max"}


model_constructor = ARDLModel
modelName = "ARDL"
model_config = {
    "constructor":{
            "lags": 5,
            "order": 5,
            "causal": True,
            "trend": "ct",
            "seasonal": False,
            "period": None,
            "missing": "drop"
        },
        "fit":{
            "cov_type": "HC0"
        }
    }

stoppingName = "wald-test"
config = {"significance_threshold": 0.05,
          "method": "wald-test",
          "max_features": 5,
          "choose_oracle": True,
          "stop_oracle": False,
          "association": association_list,
          "association.config": association_config_list,
          "model": model_constructor,
          "model.config": model_config}

df_results = run_trials(rootdir,dataset, config)

df = pd.DataFrame(df_results)
df["model"]=modelName
df["association"]=associationName
df["stopping_criterion"]=stoppingName
df.to_csv("ARDL_fmri.csv", mode="a", index=False, header=False)

File timeseries21.csv has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File timeseries7.csv has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File timeseries23.csv has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File timeseries15.csv has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File timeseries16.csv has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File timeseries18.csv has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File timeseries8.csv ha

### fMRI - ARDL - Pearson - lr-test

In [14]:
rootdir = "../"
dataset = "fMRI_processed_by_Nauta/returns/our_selection"


association_list = PearsonMultivariate
associationName = "Pearson"
association_config_list = {"return_type": "p-value",
                           "lags": 5,
                           "selection_rule": "max"}


model_constructor = ARDLModel
modelName = "ARDL"
model_config = {
    "constructor":{
            "lags": 5,
            "order": 5,
            "causal": True,
            "trend": "ct",
            "seasonal": False,
            "period": None,
            "missing": "drop"
        },
        "fit":{
            "cov_type": "HC0"
        }
    }

stoppingName = "lr-test"
config = {"significance_threshold": 0.05,
          "method": "lr-test",
          "max_features": 5,
          "choose_oracle": True,
          "stop_oracle": False,
          "association": association_list,
          "association.config": association_config_list,
          "model": model_constructor,
          "model.config": model_config}

df_results = run_trials(rootdir,dataset, config)

df = pd.DataFrame(df_results)
df["model"]=modelName
df["association"]=associationName
df["stopping_criterion"]=stoppingName
df.to_csv("ARDL_fmri.csv", mode="a", index=False, header=False)

File timeseries21.csv has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File timeseries7.csv has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File timeseries23.csv has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File timeseries15.csv has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File timeseries16.csv has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File timeseries18.csv has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File timeseries8.csv ha

### fMRI - ARDL - Spearman - lr-test

In [15]:
rootdir = "../"
dataset = "fMRI_processed_by_Nauta/returns/our_selection"


association_list = SpearmanMultivariate
associationName = "Spearman"
association_config_list = {"return_type": "p-value",
                           "lags": 5,
                           "selection_rule": "max"}


model_constructor = ARDLModel
modelName = "ARDL"
model_config = {
    "constructor":{
            "lags": 5,
            "order": 5,
            "causal": True,
            "trend": "ct",
            "seasonal": False,
            "period": None,
            "missing": "drop"
        },
        "fit":{
            "cov_type": "HC0"
        }
    }

stoppingName = "lr-test"
config = {"significance_threshold": 0.05,
          "method": "lr-test",
          "max_features": 5,
          "choose_oracle": True,
          "stop_oracle": False,
          "association": association_list,
          "association.config": association_config_list,
          "model": model_constructor,
          "model.config": model_config}

df_results = run_trials(rootdir,dataset, config)

df = pd.DataFrame(df_results)
df["model"]=modelName
df["association"]=associationName
df["stopping_criterion"]=stoppingName
df.to_csv("ARDL_fmri.csv", mode="a", index=False, header=False)

File timeseries21.csv has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File timeseries7.csv has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File timeseries23.csv has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File timeseries15.csv has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File timeseries16.csv has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File timeseries18.csv has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File timeseries8.csv ha

## CLIM

### CLIM - ARDL - Pearson - F-test

In [16]:
rootdir = "../"
dataset = "TestCLIM_N-5_T-250/returns"


association_list = PearsonMultivariate
associationName = "Pearson"
association_config_list = {"return_type": "p-value",
                           "lags": 5,
                           "selection_rule": "max"}


model_constructor = ARDLModel
modelName = "ARDL"
model_config = {
    "constructor":{
            "lags": 5,
            "order": 5,
            "causal": True,
            "trend": "ct",
            "seasonal": False,
            "period": None,
            "missing": "drop"
        },
        "fit":{
            "cov_type": "HC0"
        }
    }

stoppingName = "f-test"
config = {"significance_threshold": 0.05,
          "method": "f-test",
          "max_features": 5,
          "choose_oracle": True,
          "stop_oracle": False,
          "association": association_list,
          "association.config": association_config_list,
          "model": model_constructor,
          "model.config": model_config}

df_results = run_trials(rootdir,dataset, config)

df = pd.DataFrame(df_results)
df["model"]=modelName
df["association"]=associationName
df["stopping_criterion"]=stoppingName
df.to_csv("ARDL_clim.csv",index=False)

File TestCLIM_N-5_T-250_0089.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0074.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0149.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0055.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0154.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0110.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variabl

	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0091.txt has begun.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0072.txt has begun.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0075.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0008.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0081.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0052.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0006.txt h

	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0169.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0167.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0080.txt has begun.
	Variable 0 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0077.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0078.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0073.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variabl

File TestCLIM_N-5_T-250_0119.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0130.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0046.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0005.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0178.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0064.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variabl

### CLIM - ARDL - Spearman - F-test

In [17]:
rootdir = "../"
dataset = "TestCLIM_N-5_T-250/returns"


association_list = SpearmanMultivariate
associationName = "Spearman"
association_config_list = {"return_type": "p-value",
                           "lags": 5,
                           "selection_rule": "max"}


model_constructor = ARDLModel
modelName = "ARDL"
model_config = {
    "constructor":{
            "lags": 5,
            "order": 5,
            "causal": True,
            "trend": "ct",
            "seasonal": False,
            "period": None,
            "missing": "drop"
        },
        "fit":{
            "cov_type": "HC0"
        }
    }

stoppingName = "f-test"
config = {"significance_threshold": 0.05,
          "method": "f-test",
          "max_features": 5,
          "choose_oracle": True,
          "stop_oracle": False,
          "association": association_list,
          "association.config": association_config_list,
          "model": model_constructor,
          "model.config": model_config}

df_results = run_trials(rootdir,dataset, config)

df = pd.DataFrame(df_results)
df["model"]=modelName
df["association"]=associationName
df["stopping_criterion"]=stoppingName
df.to_csv("ARDL_clim.csv", mode="a", index=False, header=False)

File TestCLIM_N-5_T-250_0089.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0074.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0149.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0055.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0154.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0110.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variabl

	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0091.txt has begun.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0072.txt has begun.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0075.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0008.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0081.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0052.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0006.txt h

File TestCLIM_N-5_T-250_0066.txt has begun.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0169.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0167.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0080.txt has begun.
	Variable 0 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0077.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0078.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0073.txt h

	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0130.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0046.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0005.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0178.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0064.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_01

### CLIM - ARDL - Pearson - wald-test

In [18]:
rootdir = "../"
dataset = "TestCLIM_N-5_T-250/returns"


association_list = PearsonMultivariate
associationName = "Pearson"
association_config_list = {"return_type": "p-value",
                           "lags": 5,
                           "selection_rule": "max"}


model_constructor = ARDLModel
modelName = "ARDL"
model_config = {
    "constructor":{
            "lags": 5,
            "order": 5,
            "causal": True,
            "trend": "ct",
            "seasonal": False,
            "period": None,
            "missing": "drop"
        },
        "fit":{
            "cov_type": "HC0"
        }
    }

stoppingName = "wald-test"
config = {"significance_threshold": 0.05,
          "method": "wald-test",
          "max_features": 5,
          "choose_oracle": True,
          "stop_oracle": False,
          "association": association_list,
          "association.config": association_config_list,
          "model": model_constructor,
          "model.config": model_config}

df_results = run_trials(rootdir,dataset, config)

df = pd.DataFrame(df_results)
df["model"]=modelName
df["association"]=associationName
df["stopping_criterion"]=stoppingName
df.to_csv("ARDL_clim.csv", mode="a", index=False, header=False)

File TestCLIM_N-5_T-250_0089.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0074.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0149.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0055.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0154.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0110.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variabl

	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0091.txt has begun.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0072.txt has begun.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0075.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0008.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0081.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0052.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0006.txt h

File TestCLIM_N-5_T-250_0066.txt has begun.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0169.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0167.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0080.txt has begun.
	Variable 0 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0077.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0078.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0073.txt h

	Variable 1 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0119.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0130.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0046.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0005.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0178.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0064.txt has begun.
	Variabl

### CLIM - ARDL - Spearman - wald-test

In [19]:
rootdir = "../"
dataset = "TestCLIM_N-5_T-250/returns"


association_list = SpearmanMultivariate
associationName = "Spearman"
association_config_list = {"return_type": "p-value",
                           "lags": 5,
                           "selection_rule": "max"}


model_constructor = ARDLModel
modelName = "ARDL"
model_config = {
    "constructor":{
            "lags": 5,
            "order": 5,
            "causal": True,
            "trend": "ct",
            "seasonal": False,
            "period": None,
            "missing": "drop"
        },
        "fit":{
            "cov_type": "HC0"
        }
    }

stoppingName = "wald-test"
config = {"significance_threshold": 0.05,
          "method": "wald-test",
          "max_features": 5,
          "choose_oracle": True,
          "stop_oracle": False,
          "association": association_list,
          "association.config": association_config_list,
          "model": model_constructor,
          "model.config": model_config}

df_results = run_trials(rootdir,dataset, config)

df = pd.DataFrame(df_results)
df["model"]=modelName
df["association"]=associationName
df["stopping_criterion"]=stoppingName
df.to_csv("ARDL_clim.csv", mode="a", index=False, header=False)

File TestCLIM_N-5_T-250_0089.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0074.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0149.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0055.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0154.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0110.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variabl

	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0072.txt has begun.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0075.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0008.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0081.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0052.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0006.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variabl

	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0169.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0167.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0080.txt has begun.
	Variable 0 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0077.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0078.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0073.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variabl

	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0130.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0046.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0005.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0178.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0064.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0158.txt has begun.
	Variable 0 is beginning.
	Variabl

### CLIM - ARDL - Pearson - lr-test

In [20]:
rootdir = "../"
dataset = "TestCLIM_N-5_T-250/returns"


association_list = PearsonMultivariate
associationName = "Pearson"
association_config_list = {"return_type": "p-value",
                           "lags": 5,
                           "selection_rule": "max"}


model_constructor = ARDLModel
modelName = "ARDL"
model_config = {
    "constructor":{
            "lags": 5,
            "order": 5,
            "causal": True,
            "trend": "ct",
            "seasonal": False,
            "period": None,
            "missing": "drop"
        },
        "fit":{
            "cov_type": "HC0"
        }
    }

stoppingName = "lr-test"
config = {"significance_threshold": 0.05,
          "method": "lr-test",
          "max_features": 5,
          "choose_oracle": True,
          "stop_oracle": False,
          "association": association_list,
          "association.config": association_config_list,
          "model": model_constructor,
          "model.config": model_config}

df_results = run_trials(rootdir,dataset, config)

df = pd.DataFrame(df_results)
df["model"]=modelName
df["association"]=associationName
df["stopping_criterion"]=stoppingName
df.to_csv("ARDL_clim.csv", mode="a", index=False, header=False)

File TestCLIM_N-5_T-250_0089.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0074.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0149.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0055.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0154.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0110.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variabl

File TestCLIM_N-5_T-250_0091.txt has begun.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0072.txt has begun.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0075.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0008.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0081.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0052.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0006.txt has begun.
	Variable 0 is b

	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0169.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0167.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0080.txt has begun.
	Variable 0 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0077.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0078.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0073.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variabl

File TestCLIM_N-5_T-250_0119.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0130.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0046.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0005.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0178.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0064.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variabl

### CLIM - ARDL - Spearman - lr-test

In [21]:
rootdir = "../"
dataset = "TestCLIM_N-5_T-250/returns"


association_list = SpearmanMultivariate
associationName = "Spearman"
association_config_list = {"return_type": "p-value",
                           "lags": 5,
                           "selection_rule": "max"}


model_constructor = ARDLModel
modelName = "ARDL"
model_config = {
    "constructor":{
            "lags": 5,
            "order": 5,
            "causal": True,
            "trend": "ct",
            "seasonal": False,
            "period": None,
            "missing": "drop"
        },
        "fit":{
            "cov_type": "HC0"
        }
    }

stoppingName = "lr-test"
config = {"significance_threshold": 0.05,
          "method": "lr-test",
          "max_features": 5,
          "choose_oracle": True,
          "stop_oracle": False,
          "association": association_list,
          "association.config": association_config_list,
          "model": model_constructor,
          "model.config": model_config}

df_results = run_trials(rootdir,dataset, config)

df = pd.DataFrame(df_results)
df["model"]=modelName
df["association"]=associationName
df["stopping_criterion"]=stoppingName
df.to_csv("ARDL_clim.csv", mode="a", index=False, header=False)

File TestCLIM_N-5_T-250_0089.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0074.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0149.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0055.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0154.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0110.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variabl

	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0072.txt has begun.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0075.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0008.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0081.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0052.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0006.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variabl

	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0167.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0080.txt has begun.
	Variable 0 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0077.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0078.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0073.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0122.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variabl

File TestCLIM_N-5_T-250_0130.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0046.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0005.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0178.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0064.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
File TestCLIM_N-5_T-250_0158.txt has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variabl

## Finance

### Finance - ARDL - Pearson - F-test

In [22]:
rootdir = "../"
dataset = "FinanceCPT/returns/our_selection"


association_list = PearsonMultivariate
associationName = "Pearson"
association_config_list = {"return_type": "p-value",
                           "lags": 10,
                           "selection_rule": "max"}


model_constructor = ARDLModel
modelName = "ARDL"
model_config = {
    "constructor":{
            "lags": 10,
            "order": 10,
            "causal": True,
            "trend": "ct",
            "seasonal": False,
            "period": None,
            "missing": "drop"
        },
        "fit":{
            "cov_type": "HC0"
        }
    }

stoppingName = "f-test"
config = {"significance_threshold": 0.05,
          "method": "f-test",
          "max_features": 15,
          "choose_oracle": True,
          "stop_oracle": False,
          "association": association_list,
          "association.config": association_config_list,
          "model": model_constructor,
          "model.config": model_config}

df_results = run_trials(rootdir,dataset, config)

df = pd.DataFrame(df_results)
df["model"]=modelName
df["association"]=associationName
df["stopping_criterion"]=stoppingName
df.to_csv("ARDL_finance.csv",index=False)

File random-rels_20_1C_returns30007000.csv has begun.
	Variable 2 is beginning.
	Variable 4 is beginning.
	Variable 5 is beginning.
	Variable 6 is beginning.
	Variable 7 is beginning.
	Variable 11 is beginning.
	Variable 13 is beginning.
	Variable 16 is beginning.
	Variable 17 is beginning.
	Variable 18 is beginning.
	Variable 22 is beginning.
	Variable 23 is beginning.
	Variable 24 is beginning.
File random-rels_20_1E_returns30007000.csv has begun.
	Variable 1 is beginning.
	Variable 3 is beginning.
	Variable 5 is beginning.
	Variable 9 is beginning.
	Variable 10 is beginning.
	Variable 12 is beginning.
	Variable 14 is beginning.
	Variable 16 is beginning.
	Variable 17 is beginning.
	Variable 19 is beginning.
	Variable 20 is beginning.
	Variable 22 is beginning.
	Variable 23 is beginning.
File random-rels_20_1_3_returns800012000.csv has begun.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 5 is beginning.
	Variable 6 is beginning.
	Variable 7 i

### Finance - ARDL - Spearman - F-test

In [23]:
rootdir = "../"
dataset = "FinanceCPT/returns/our_selection"


association_list = SpearmanMultivariate
associationName = "Spearman"
association_config_list = {"return_type": "p-value",
                           "lags": 10,
                           "selection_rule": "max"}


model_constructor = ARDLModel
modelName = "ARDL"
model_config = {
    "constructor":{
            "lags": 10,
            "order": 10,
            "causal": True,
            "trend": "ct",
            "seasonal": False,
            "period": None,
            "missing": "drop"
        },
        "fit":{
            "cov_type": "HC0"
        }
    }

stoppingName = "f-test"
config = {"significance_threshold": 0.05,
          "method": "f-test",
          "max_features": 15,
          "choose_oracle": True,
          "stop_oracle": False,
          "association": association_list,
          "association.config": association_config_list,
          "model": model_constructor,
          "model.config": model_config}

df_results = run_trials(rootdir,dataset, config)

df = pd.DataFrame(df_results)
df["model"]=modelName
df["association"]=associationName
df["stopping_criterion"]=stoppingName
df.to_csv("ARDL_finance.csv", mode="a", index=False, header=False)

File random-rels_20_1C_returns30007000.csv has begun.
	Variable 2 is beginning.
	Variable 4 is beginning.
	Variable 5 is beginning.
	Variable 6 is beginning.
	Variable 7 is beginning.
	Variable 11 is beginning.
	Variable 13 is beginning.
	Variable 16 is beginning.
	Variable 17 is beginning.
	Variable 18 is beginning.
	Variable 22 is beginning.
	Variable 23 is beginning.
	Variable 24 is beginning.
File random-rels_20_1E_returns30007000.csv has begun.
	Variable 1 is beginning.
	Variable 3 is beginning.
	Variable 5 is beginning.
	Variable 9 is beginning.
	Variable 10 is beginning.
	Variable 12 is beginning.
	Variable 14 is beginning.
	Variable 16 is beginning.
	Variable 17 is beginning.
	Variable 19 is beginning.
	Variable 20 is beginning.
	Variable 22 is beginning.
	Variable 23 is beginning.
File random-rels_20_1_3_returns800012000.csv has begun.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 5 is beginning.
	Variable 6 is beginning.
	Variable 7 i

### Finance - ARDL - Pearson - wald-test

In [24]:
rootdir = "../"
dataset = "FinanceCPT/returns/our_selection"


association_list = PearsonMultivariate
associationName = "Pearson"
association_config_list = {"return_type": "p-value",
                           "lags": 10,
                           "selection_rule": "max"}


model_constructor = ARDLModel
modelName = "ARDL"
model_config = {
    "constructor":{
            "lags": 10,
            "order": 10,
            "causal": True,
            "trend": "ct",
            "seasonal": False,
            "period": None,
            "missing": "drop"
        },
        "fit":{
            "cov_type": "HC0"
        }
    }

stoppingName = "wald-test"
config = {"significance_threshold": 0.05,
          "method": "wald-test",
          "max_features": 15,
          "choose_oracle": True,
          "stop_oracle": False,
          "association": association_list,
          "association.config": association_config_list,
          "model": model_constructor,
          "model.config": model_config}

df_results = run_trials(rootdir,dataset, config)

df = pd.DataFrame(df_results)
df["model"]=modelName
df["association"]=associationName
df["stopping_criterion"]=stoppingName
df.to_csv("ARDL_finance.csv", mode="a", index=False, header=False)

File random-rels_20_1C_returns30007000.csv has begun.
	Variable 2 is beginning.
	Variable 4 is beginning.
	Variable 5 is beginning.
	Variable 6 is beginning.
	Variable 7 is beginning.
	Variable 11 is beginning.
	Variable 13 is beginning.
	Variable 16 is beginning.
	Variable 17 is beginning.
	Variable 18 is beginning.
	Variable 22 is beginning.
	Variable 23 is beginning.
	Variable 24 is beginning.
File random-rels_20_1E_returns30007000.csv has begun.
	Variable 1 is beginning.
	Variable 3 is beginning.
	Variable 5 is beginning.
	Variable 9 is beginning.
	Variable 10 is beginning.
	Variable 12 is beginning.
	Variable 14 is beginning.
	Variable 16 is beginning.
	Variable 17 is beginning.
	Variable 19 is beginning.
	Variable 20 is beginning.
	Variable 22 is beginning.
	Variable 23 is beginning.
File random-rels_20_1_3_returns800012000.csv has begun.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 5 is beginning.
	Variable 6 is beginning.
	Variable 7 i

### Finance - ARDL - Spearman - wald-test

In [25]:
rootdir = "../"
dataset = "FinanceCPT/returns/our_selection"


association_list = SpearmanMultivariate
associationName = "Spearman"
association_config_list = {"return_type": "p-value",
                           "lags": 10,
                           "selection_rule": "max"}


model_constructor = ARDLModel
modelName = "ARDL"
model_config = {
    "constructor":{
            "lags": 10,
            "order": 10,
            "causal": True,
            "trend": "ct",
            "seasonal": False,
            "period": None,
            "missing": "drop"
        },
        "fit":{
            "cov_type": "HC0"
        }
    }

stoppingName = "wald-test"
config = {"significance_threshold": 0.05,
          "method": "wald-test",
          "max_features": 15,
          "choose_oracle": True,
          "stop_oracle": False,
          "association": association_list,
          "association.config": association_config_list,
          "model": model_constructor,
          "model.config": model_config}

df_results = run_trials(rootdir,dataset, config)

df = pd.DataFrame(df_results)
df["model"]=modelName
df["association"]=associationName
df["stopping_criterion"]=stoppingName
df.to_csv("ARDL_finance.csv", mode="a", index=False, header=False)

File random-rels_20_1C_returns30007000.csv has begun.
	Variable 2 is beginning.
	Variable 4 is beginning.
	Variable 5 is beginning.
	Variable 6 is beginning.
	Variable 7 is beginning.
	Variable 11 is beginning.
	Variable 13 is beginning.
	Variable 16 is beginning.
	Variable 17 is beginning.
	Variable 18 is beginning.
	Variable 22 is beginning.
	Variable 23 is beginning.
	Variable 24 is beginning.
File random-rels_20_1E_returns30007000.csv has begun.
	Variable 1 is beginning.
	Variable 3 is beginning.
	Variable 5 is beginning.
	Variable 9 is beginning.
	Variable 10 is beginning.
	Variable 12 is beginning.
	Variable 14 is beginning.
	Variable 16 is beginning.
	Variable 17 is beginning.
	Variable 19 is beginning.
	Variable 20 is beginning.
	Variable 22 is beginning.
	Variable 23 is beginning.
File random-rels_20_1_3_returns800012000.csv has begun.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 5 is beginning.
	Variable 6 is beginning.
	Variable 7 i

### Finance - ARDL - Pearson - lr-test

In [26]:
rootdir = "../"
dataset = "FinanceCPT/returns/our_selection"


association_list = PearsonMultivariate
associationName = "Pearson"
association_config_list = {"return_type": "p-value",
                           "lags": 10,
                           "selection_rule": "max"}


model_constructor = ARDLModel
modelName = "ARDL"
model_config = {
    "constructor":{
            "lags": 10,
            "order": 10,
            "causal": True,
            "trend": "ct",
            "seasonal": False,
            "period": None,
            "missing": "drop"
        },
        "fit":{
            "cov_type": "HC0"
        }
    }

stoppingName = "lr-test"
config = {"significance_threshold": 0.05,
          "method": "lr-test",
          "max_features": 15,
          "choose_oracle": True,
          "stop_oracle": False,
          "association": association_list,
          "association.config": association_config_list,
          "model": model_constructor,
          "model.config": model_config}

df_results = run_trials(rootdir,dataset, config)

df = pd.DataFrame(df_results)
df["model"]=modelName
df["association"]=associationName
df["stopping_criterion"]=stoppingName
df.to_csv("ARDL_finance.csv", mode="a", index=False, header=False)

File random-rels_20_1C_returns30007000.csv has begun.
	Variable 2 is beginning.
	Variable 4 is beginning.
	Variable 5 is beginning.
	Variable 6 is beginning.
	Variable 7 is beginning.
	Variable 11 is beginning.
	Variable 13 is beginning.
	Variable 16 is beginning.
	Variable 17 is beginning.
	Variable 18 is beginning.
	Variable 22 is beginning.
	Variable 23 is beginning.
	Variable 24 is beginning.
File random-rels_20_1E_returns30007000.csv has begun.
	Variable 1 is beginning.
	Variable 3 is beginning.
	Variable 5 is beginning.
	Variable 9 is beginning.
	Variable 10 is beginning.
	Variable 12 is beginning.
	Variable 14 is beginning.
	Variable 16 is beginning.
	Variable 17 is beginning.
	Variable 19 is beginning.
	Variable 20 is beginning.
	Variable 22 is beginning.
	Variable 23 is beginning.
File random-rels_20_1_3_returns800012000.csv has begun.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 5 is beginning.
	Variable 6 is beginning.
	Variable 7 i

### Finance - ARDL - Spearman - lr-test

In [27]:
rootdir = "../"
dataset = "FinanceCPT/returns/our_selection"


association_list = SpearmanMultivariate
associationName = "Spearman"
association_config_list = {"return_type": "p-value",
                           "lags": 10,
                           "selection_rule": "max"}


model_constructor = ARDLModel
modelName = "ARDL"
model_config = {
    "constructor":{
            "lags": 10,
            "order": 10,
            "causal": True,
            "trend": "ct",
            "seasonal": False,
            "period": None,
            "missing": "drop"
        },
        "fit":{
            "cov_type": "HC0"
        }
    }

stoppingName = "lr-test"
config = {"significance_threshold": 0.05,
          "method": "lr-test",
          "max_features": 15,
          "choose_oracle": True,
          "stop_oracle": False,
          "association": association_list,
          "association.config": association_config_list,
          "model": model_constructor,
          "model.config": model_config}

df_results = run_trials(rootdir,dataset, config)

df = pd.DataFrame(df_results)
df["model"]=modelName
df["association"]=associationName
df["stopping_criterion"]=stoppingName
df.to_csv("ARDL_finance.csv", mode="a", index=False, header=False)

File random-rels_20_1C_returns30007000.csv has begun.
	Variable 2 is beginning.
	Variable 4 is beginning.
	Variable 5 is beginning.
	Variable 6 is beginning.
	Variable 7 is beginning.
	Variable 11 is beginning.
	Variable 13 is beginning.
	Variable 16 is beginning.
	Variable 17 is beginning.
	Variable 18 is beginning.
	Variable 22 is beginning.
	Variable 23 is beginning.
	Variable 24 is beginning.
File random-rels_20_1E_returns30007000.csv has begun.
	Variable 1 is beginning.
	Variable 3 is beginning.
	Variable 5 is beginning.
	Variable 9 is beginning.
	Variable 10 is beginning.
	Variable 12 is beginning.
	Variable 14 is beginning.
	Variable 16 is beginning.
	Variable 17 is beginning.
	Variable 19 is beginning.
	Variable 20 is beginning.
	Variable 22 is beginning.
	Variable 23 is beginning.
File random-rels_20_1_3_returns800012000.csv has begun.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 5 is beginning.
	Variable 6 is beginning.
	Variable 7 i

## VarProcess

### VAR10 - ARDL - Pearson - F-test

In [28]:
rootdir = "../"
dataset = "VARSmall/returns"


association_list = PearsonMultivariate
associationName = "Pearson"
association_config_list = {"return_type": "p-value",
                           "lags": 10,
                           "selection_rule": "max"}


model_constructor = ARDLModel
modelName = "ARDL"
model_config = {
    "constructor":{
            "lags": 10,
            "order": 10,
            "causal": True,
            "trend": "ct",
            "seasonal": False,
            "period": None,
            "missing": "drop"
        },
        "fit":{
            "cov_type": "HC0"
        }
    }

stoppingName = "f-test"
config = {"significance_threshold": 0.05,
          "method": "f-test",
          "max_features": 10,
          "choose_oracle": True,
          "stop_oracle": False,
          "association": association_list,
          "association.config": association_config_list,
          "model": model_constructor,
          "model.config": model_config}

df_results = run_trials(rootdir,dataset, config)

df = pd.DataFrame(df_results)
df["model"]=modelName
df["association"]=associationName
df["stopping_criterion"]=stoppingName
df.to_csv("ARDL_var10.csv",index=False)

File data_1.csv has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
	Variable 5 is beginning.
	Variable 6 is beginning.
	Variable 7 is beginning.
	Variable 8 is beginning.
	Variable 9 is beginning.


### VAR10 - ARDL - Spearman - F-test

In [29]:
rootdir = "../"
dataset = "VARSmall/returns"


association_list = SpearmanMultivariate
associationName = "Spearman"
association_config_list = {"return_type": "p-value",
                           "lags": 10,
                           "selection_rule": "max"}


model_constructor = ARDLModel
modelName = "ARDL"
model_config = {
    "constructor":{
            "lags": 10,
            "order": 10,
            "causal": True,
            "trend": "ct",
            "seasonal": False,
            "period": None,
            "missing": "drop"
        },
        "fit":{
            "cov_type": "HC0"
        }
    }

stoppingName = "f-test"
config = {"significance_threshold": 0.05,
          "method": "f-test",
          "max_features": 10,
          "choose_oracle": True,
          "stop_oracle": False,
          "association": association_list,
          "association.config": association_config_list,
          "model": model_constructor,
          "model.config": model_config}

df_results = run_trials(rootdir,dataset, config)

df = pd.DataFrame(df_results)
df["model"]=modelName
df["association"]=associationName
df["stopping_criterion"]=stoppingName
df.to_csv("ARDL_var10.csv", mode="a", index=False, header=False)

File data_1.csv has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
	Variable 5 is beginning.
	Variable 6 is beginning.
	Variable 7 is beginning.
	Variable 8 is beginning.
	Variable 9 is beginning.


### VAR10 - ARDL - Pearson - wald-test

In [30]:
rootdir = "../"
dataset = "VARSmall/returns"


association_list = PearsonMultivariate
associationName = "Pearson"
association_config_list = {"return_type": "p-value",
                           "lags": 10,
                           "selection_rule": "max"}


model_constructor = ARDLModel
modelName = "ARDL"
model_config = {
    "constructor":{
            "lags": 10,
            "order": 10,
            "causal": True,
            "trend": "ct",
            "seasonal": False,
            "period": None,
            "missing": "drop"
        },
        "fit":{
            "cov_type": "HC0"
        }
    }

stoppingName = "wald-test"
config = {"significance_threshold": 0.05,
          "method": "wald-test",
          "max_features": 10,
          "choose_oracle": True,
          "stop_oracle": False,
          "association": association_list,
          "association.config": association_config_list,
          "model": model_constructor,
          "model.config": model_config}

df_results = run_trials(rootdir,dataset, config)

df = pd.DataFrame(df_results)
df["model"]=modelName
df["association"]=associationName
df["stopping_criterion"]=stoppingName
df.to_csv("ARDL_var10.csv", mode="a", index=False, header=False)

File data_1.csv has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
	Variable 5 is beginning.
	Variable 6 is beginning.
	Variable 7 is beginning.
	Variable 8 is beginning.
	Variable 9 is beginning.


### VAR10 - ARDL - Spearman - wald-test

In [31]:
rootdir = "../"
dataset = "VARSmall/returns"


association_list = SpearmanMultivariate
associationName = "Spearman"
association_config_list = {"return_type": "p-value",
                           "lags": 10,
                           "selection_rule": "max"}


model_constructor = ARDLModel
modelName = "ARDL"
model_config = {
    "constructor":{
            "lags": 10,
            "order": 10,
            "causal": True,
            "trend": "ct",
            "seasonal": False,
            "period": None,
            "missing": "drop"
        },
        "fit":{
            "cov_type": "HC0"
        }
    }

stoppingName = "wald-test"
config = {"significance_threshold": 0.05,
          "method": "wald-test",
          "max_features": 10,
          "choose_oracle": True,
          "stop_oracle": False,
          "association": association_list,
          "association.config": association_config_list,
          "model": model_constructor,
          "model.config": model_config}

df_results = run_trials(rootdir,dataset, config)

df = pd.DataFrame(df_results)
df["model"]=modelName
df["association"]=associationName
df["stopping_criterion"]=stoppingName
df.to_csv("ARDL_var10.csv", mode="a", index=False, header=False)

File data_1.csv has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
	Variable 5 is beginning.
	Variable 6 is beginning.
	Variable 7 is beginning.
	Variable 8 is beginning.
	Variable 9 is beginning.


### VAR10 - ARDL - Pearson - lr-test

In [32]:
rootdir = "../"
dataset = "VARSmall/returns"


association_list = PearsonMultivariate
associationName = "Pearson"
association_config_list = {"return_type": "p-value",
                           "lags": 10,
                           "selection_rule": "max"}


model_constructor = ARDLModel
modelName = "ARDL"
model_config = {
    "constructor":{
            "lags": 10,
            "order": 10,
            "causal": True,
            "trend": "ct",
            "seasonal": False,
            "period": None,
            "missing": "drop"
        },
        "fit":{
            "cov_type": "HC0"
        }
    }

stoppingName = "lr-test"
config = {"significance_threshold": 0.05,
          "method": "lr-test",
          "max_features": 10,
          "choose_oracle": True,
          "stop_oracle": False,
          "association": association_list,
          "association.config": association_config_list,
          "model": model_constructor,
          "model.config": model_config}

df_results = run_trials(rootdir,dataset, config)

df = pd.DataFrame(df_results)
df["model"]=modelName
df["association"]=associationName
df["stopping_criterion"]=stoppingName
df.to_csv("ARDL_var10.csv", mode="a", index=False, header=False)

File data_1.csv has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
	Variable 5 is beginning.
	Variable 6 is beginning.
	Variable 7 is beginning.
	Variable 8 is beginning.
	Variable 9 is beginning.


### VAR10 - ARDL - Spearman - lr-test

In [33]:
rootdir = "../"
dataset = "VARSmall/returns"


association_list = SpearmanMultivariate
associationName = "Spearman"
association_config_list = {"return_type": "p-value",
                           "lags": 10,
                           "selection_rule": "max"}


model_constructor = ARDLModel
modelName = "ARDL"
model_config = {
    "constructor":{
            "lags": 10,
            "order": 10,
            "causal": True,
            "trend": "ct",
            "seasonal": False,
            "period": None,
            "missing": "drop"
        },
        "fit":{
            "cov_type": "HC0"
        }
    }

stoppingName = "lr-test"
config = {"significance_threshold": 0.05,
          "method": "lr-test",
          "max_features": 10,
          "choose_oracle": True,
          "stop_oracle": False,
          "association": association_list,
          "association.config": association_config_list,
          "model": model_constructor,
          "model.config": model_config}

df_results = run_trials(rootdir,dataset, config)

df = pd.DataFrame(df_results)
df["model"]=modelName
df["association"]=associationName
df["stopping_criterion"]=stoppingName
df.to_csv("ARDL_var10.csv", mode="a", index=False, header=False)

File data_1.csv has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
	Variable 5 is beginning.
	Variable 6 is beginning.
	Variable 7 is beginning.
	Variable 8 is beginning.
	Variable 9 is beginning.


## VARLarge

### VAR10000 - ARDL - Pearson - F-test

In [34]:
rootdir = "../"
dataset = "VARLarge/returns"


association_list = PearsonMultivariate
associationName = "Pearson"
association_config_list = {"return_type": "p-value",
                           "lags": 10,
                           "selection_rule": "max"}


model_constructor = ARDLModel
modelName = "ARDL"
model_config = {
    "constructor":{
            "lags": 10,
            "order": 10,
            "causal": True,
            "trend": "ct",
            "seasonal": False,
            "period": None,
            "missing": "drop"
        },
        "fit":{
            "cov_type": "HC0"
        }
    }

stoppingName = "f-test"
config = {"significance_threshold": 0.05,
          "method": "f-test",
          "max_features": 20,
          "choose_oracle": True,
          "stop_oracle": False,
          "association": association_list,
          "association.config": association_config_list,
          "model": model_constructor,
          "model.config": model_config}

df_results = run_trials(rootdir,dataset, config,
                        select_random_targets = True, number_selections = 20)

df = pd.DataFrame(df_results)
df["model"]=modelName
df["association"]=associationName
df["stopping_criterion"]=stoppingName
df.to_csv("ARDL_var10000.csv",index=False)

File data_0.csv has begun.
	Variable 8506 is beginning.
	Variable 6369 is beginning.
	Variable 5111 is beginning.
	Variable 2697 is beginning.
	Variable 3078 is beginning.
	Variable 409 is beginning.
	Variable 752 is beginning.
	Variable 165 is beginning.
	Variable 1752 is beginning.
	Variable 8132 is beginning.
	Variable 6494 is beginning.
	Variable 9127 is beginning.
	Variable 5036 is beginning.
	Variable 6066 is beginning.
	Variable 9707 is beginning.
	Variable 7294 is beginning.
	Variable 6322 is beginning.
	Variable 5436 is beginning.
	Variable 5599 is beginning.
	Variable 9350 is beginning.


### VAR10000 - ARDL - Spearman - F-test

In [35]:
rootdir = "../"
dataset = "VARLarge/returns"


association_list = SpearmanMultivariate
associationName = "Spearman"
association_config_list = {"return_type": "p-value",
                           "lags": 10,
                           "selection_rule": "max"}


model_constructor = ARDLModel
modelName = "ARDL"
model_config = {
    "constructor":{
            "lags": 10,
            "order": 10,
            "causal": True,
            "trend": "ct",
            "seasonal": False,
            "period": None,
            "missing": "drop"
        },
        "fit":{
            "cov_type": "HC0"
        }
    }

stoppingName = "f-test"
config = {"significance_threshold": 0.05,
          "method": "f-test",
          "max_features": 20,
          "choose_oracle": True,
          "stop_oracle": False,
          "association": association_list,
          "association.config": association_config_list,
          "model": model_constructor,
          "model.config": model_config}

df_results = run_trials(rootdir,dataset, config,
                        select_random_targets = True, number_selections = 20)

df = pd.DataFrame(df_results)
df["model"]=modelName
df["association"]=associationName
df["stopping_criterion"]=stoppingName
df.to_csv("ARDL_var10000.csv", mode="a", index=False, header=False)

File data_0.csv has begun.
	Variable 8506 is beginning.
	Variable 6369 is beginning.
	Variable 5111 is beginning.
	Variable 2697 is beginning.
	Variable 3078 is beginning.
	Variable 409 is beginning.
	Variable 752 is beginning.
	Variable 165 is beginning.
	Variable 1752 is beginning.
	Variable 8132 is beginning.
	Variable 6494 is beginning.
	Variable 9127 is beginning.
	Variable 5036 is beginning.
	Variable 6066 is beginning.
	Variable 9707 is beginning.
	Variable 7294 is beginning.
	Variable 6322 is beginning.
	Variable 5436 is beginning.
	Variable 5599 is beginning.
	Variable 9350 is beginning.


### VAR10000 - ARDL - Pearson - wald-test

In [36]:
rootdir = "../"
dataset = "VARLarge/returns"


association_list = PearsonMultivariate
associationName = "Pearson"
association_config_list = {"return_type": "p-value",
                           "lags": 10,
                           "selection_rule": "max"}


model_constructor = ARDLModel
modelName = "ARDL"
model_config = {
    "constructor":{
            "lags": 10,
            "order": 10,
            "causal": True,
            "trend": "ct",
            "seasonal": False,
            "period": None,
            "missing": "drop"
        },
        "fit":{
            "cov_type": "HC0"
        }
    }

stoppingName = "wald-test"
config = {"significance_threshold": 0.05,
          "method": "wald-test",
          "max_features": 20,
          "choose_oracle": True,
          "stop_oracle": False,
          "association": association_list,
          "association.config": association_config_list,
          "model": model_constructor,
          "model.config": model_config}

df_results = run_trials(rootdir,dataset, config,
                        select_random_targets = True, number_selections = 20)

df = pd.DataFrame(df_results)
df["model"]=modelName
df["association"]=associationName
df["stopping_criterion"]=stoppingName
df.to_csv("ARDL_var10000.csv", mode="a", index=False, header=False)

File data_0.csv has begun.
	Variable 8506 is beginning.
	Variable 6369 is beginning.
	Variable 5111 is beginning.
	Variable 2697 is beginning.
	Variable 3078 is beginning.
	Variable 409 is beginning.
	Variable 752 is beginning.
	Variable 165 is beginning.
	Variable 1752 is beginning.
	Variable 8132 is beginning.
	Variable 6494 is beginning.
	Variable 9127 is beginning.
	Variable 5036 is beginning.
	Variable 6066 is beginning.
	Variable 9707 is beginning.
	Variable 7294 is beginning.
	Variable 6322 is beginning.
	Variable 5436 is beginning.
	Variable 5599 is beginning.
	Variable 9350 is beginning.


### VAR10000 - ARDL - Spearman - wald-test

In [37]:
rootdir = "../"
dataset = "VARLarge/returns"


association_list = SpearmanMultivariate
associationName = "Spearman"
association_config_list = {"return_type": "p-value",
                           "lags": 10,
                           "selection_rule": "max"}


model_constructor = ARDLModel
modelName = "ARDL"
model_config = {
    "constructor":{
            "lags": 10,
            "order": 10,
            "causal": True,
            "trend": "ct",
            "seasonal": False,
            "period": None,
            "missing": "drop"
        },
        "fit":{
            "cov_type": "HC0"
        }
    }

stoppingName = "wald-test"
config = {"significance_threshold": 0.05,
          "method": "wald-test",
          "max_features": 20,
          "choose_oracle": True,
          "stop_oracle": False,
          "association": association_list,
          "association.config": association_config_list,
          "model": model_constructor,
          "model.config": model_config}

df_results = run_trials(rootdir,dataset, config,
                        select_random_targets = True, number_selections = 20)

df = pd.DataFrame(df_results)
df["model"]=modelName
df["association"]=associationName
df["stopping_criterion"]=stoppingName
df.to_csv("ARDL_var10000.csv", mode="a", index=False, header=False)

File data_0.csv has begun.
	Variable 8506 is beginning.
	Variable 6369 is beginning.
	Variable 5111 is beginning.
	Variable 2697 is beginning.
	Variable 3078 is beginning.
	Variable 409 is beginning.
	Variable 752 is beginning.
	Variable 165 is beginning.
	Variable 1752 is beginning.
	Variable 8132 is beginning.
	Variable 6494 is beginning.
	Variable 9127 is beginning.
	Variable 5036 is beginning.
	Variable 6066 is beginning.
	Variable 9707 is beginning.
	Variable 7294 is beginning.
	Variable 6322 is beginning.
	Variable 5436 is beginning.
	Variable 5599 is beginning.
	Variable 9350 is beginning.


### VAR10000 - ARDL - Pearson - lr-test

In [38]:
rootdir = "../"
dataset = "VARLarge/returns"


association_list = PearsonMultivariate
associationName = "Pearson"
association_config_list = {"return_type": "p-value",
                           "lags": 10,
                           "selection_rule": "max"}


model_constructor = ARDLModel
modelName = "ARDL"
model_config = {
    "constructor":{
            "lags": 10,
            "order": 10,
            "causal": True,
            "trend": "ct",
            "seasonal": False,
            "period": None,
            "missing": "drop"
        },
        "fit":{
            "cov_type": "HC0"
        }
    }

stoppingName = "lr-test"
config = {"significance_threshold": 0.05,
          "method": "lr-test",
          "max_features": 20,
          "choose_oracle": True,
          "stop_oracle": False,
          "association": association_list,
          "association.config": association_config_list,
          "model": model_constructor,
          "model.config": model_config}

df_results = run_trials(rootdir,dataset, config,
                        select_random_targets = True, number_selections = 20)

df = pd.DataFrame(df_results)
df["model"]=modelName
df["association"]=associationName
df["stopping_criterion"]=stoppingName
df.to_csv("ARDL_var10000.csv", mode="a", index=False, header=False)

File data_0.csv has begun.
	Variable 8506 is beginning.
	Variable 6369 is beginning.
	Variable 5111 is beginning.
	Variable 2697 is beginning.
	Variable 3078 is beginning.
	Variable 409 is beginning.
	Variable 752 is beginning.
	Variable 165 is beginning.
	Variable 1752 is beginning.
	Variable 8132 is beginning.
	Variable 6494 is beginning.
	Variable 9127 is beginning.
	Variable 5036 is beginning.
	Variable 6066 is beginning.
	Variable 9707 is beginning.
	Variable 7294 is beginning.
	Variable 6322 is beginning.
	Variable 5436 is beginning.
	Variable 5599 is beginning.
	Variable 9350 is beginning.


### VAR10000 - ARDL - Spearman - lr-test

In [39]:
rootdir = "../"
dataset = "VARLarge/returns"


association_list = SpearmanMultivariate
associationName = "Spearman"
association_config_list = {"return_type": "p-value",
                           "lags": 10,
                           "selection_rule": "max"}


model_constructor = ARDLModel
modelName = "ARDL"
model_config = {
    "constructor":{
            "lags": 10,
            "order": 10,
            "causal": True,
            "trend": "ct",
            "seasonal": False,
            "period": None,
            "missing": "drop"
        },
        "fit":{
            "cov_type": "HC0"
        }
    }

stoppingName = "lr-test"
config = {"significance_threshold": 0.05,
          "method": "lr-test",
          "max_features": 20,
          "choose_oracle": True,
          "stop_oracle": False,
          "association": association_list,
          "association.config": association_config_list,
          "model": model_constructor,
          "model.config": model_config}

df_results = run_trials(rootdir,dataset, config,
                        select_random_targets = True, number_selections = 20)

df = pd.DataFrame(df_results)
df["model"]=modelName
df["association"]=associationName
df["stopping_criterion"]=stoppingName
df.to_csv("ARDL_var10000.csv", mode="a", index=False, header=False)

File data_0.csv has begun.
	Variable 8506 is beginning.
	Variable 6369 is beginning.
	Variable 5111 is beginning.
	Variable 2697 is beginning.
	Variable 3078 is beginning.
	Variable 409 is beginning.
	Variable 752 is beginning.
	Variable 165 is beginning.
	Variable 1752 is beginning.
	Variable 8132 is beginning.
	Variable 6494 is beginning.
	Variable 9127 is beginning.
	Variable 5036 is beginning.
	Variable 6066 is beginning.
	Variable 9707 is beginning.
	Variable 7294 is beginning.
	Variable 6322 is beginning.
	Variable 5436 is beginning.
	Variable 5599 is beginning.
	Variable 9350 is beginning.


## VARVaried

### VARVaried - ARDL - Pearson - F-test

In [15]:
rootdir = "../"
dataset = "VARVaried/returns"


association_list = PearsonMultivariate
associationName = "Pearson"
association_config_list = {"return_type": "p-value",
                           "lags": 10,
                           "selection_rule": "max"}


model_constructor = ARDLModel
modelName = "ARDL"
model_config = {
    "constructor":{
            "lags": 10,
            "order": 10,
            "causal": True,
            "trend": "ct",
            "seasonal": False,
            "period": None,
            "missing": "drop"
        },
        "fit":{
            "cov_type": "HC0"
        }
    }

stoppingName = "f-test"
config = {"significance_threshold": 0.05,
          "method": "f-test",
          "max_features": 10,
          "choose_oracle": True,
          "stop_oracle": False,
          "association": association_list,
          "association.config": association_config_list,
          "model": model_constructor,
          "model.config": model_config}

df_results = run_trials(rootdir,dataset, config,
                        select_random_targets = False, number_selections = 20)

df = pd.DataFrame(df_results)
df["model"]=modelName
df["association"]=associationName
df["stopping_criterion"]=stoppingName
df.to_csv("ARDL_varvaried.csv",index=False)

File data_1.csv has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
	Variable 5 is beginning.
	Variable 6 is beginning.
	Variable 7 is beginning.
	Variable 8 is beginning.
	Variable 9 is beginning.
File data_11.csv has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
	Variable 5 is beginning.
	Variable 6 is beginning.
	Variable 7 is beginning.
	Variable 8 is beginning.
	Variable 9 is beginning.
	Variable 10 is beginning.
	Variable 11 is beginning.
	Variable 12 is beginning.
	Variable 13 is beginning.
	Variable 14 is beginning.
	Variable 15 is beginning.
	Variable 16 is beginning.
	Variable 17 is beginning.
	Variable 18 is beginning.
	Variable 19 is beginning.
	Variable 20 is beginning.
	Variable 21 is beginning.
	Variable 22 is beginning.
	Variable 23 is beginning.
	Variable 24 is beginning.
	Variable 25 is begi

	Variable 216 is beginning.
	Variable 217 is beginning.
	Variable 218 is beginning.
	Variable 219 is beginning.
	Variable 220 is beginning.
	Variable 221 is beginning.
	Variable 222 is beginning.
	Variable 223 is beginning.
	Variable 224 is beginning.
	Variable 225 is beginning.
	Variable 226 is beginning.
	Variable 227 is beginning.
	Variable 228 is beginning.
	Variable 229 is beginning.
	Variable 230 is beginning.
	Variable 231 is beginning.
	Variable 232 is beginning.
	Variable 233 is beginning.
	Variable 234 is beginning.
	Variable 235 is beginning.
	Variable 236 is beginning.
	Variable 237 is beginning.
	Variable 238 is beginning.
	Variable 239 is beginning.
	Variable 240 is beginning.
	Variable 241 is beginning.
	Variable 242 is beginning.
	Variable 243 is beginning.
	Variable 244 is beginning.
	Variable 245 is beginning.
	Variable 246 is beginning.
	Variable 247 is beginning.
	Variable 248 is beginning.
	Variable 249 is beginning.
	Variable 250 is beginning.
	Variable 251 is beg

	Variable 33 is beginning.
	Variable 34 is beginning.
	Variable 35 is beginning.
	Variable 36 is beginning.
	Variable 37 is beginning.
	Variable 38 is beginning.
	Variable 39 is beginning.
	Variable 40 is beginning.
	Variable 41 is beginning.
	Variable 42 is beginning.
	Variable 43 is beginning.
	Variable 44 is beginning.
	Variable 45 is beginning.
	Variable 46 is beginning.
	Variable 47 is beginning.
	Variable 48 is beginning.
	Variable 49 is beginning.
	Variable 50 is beginning.
	Variable 51 is beginning.
	Variable 52 is beginning.
	Variable 53 is beginning.
	Variable 54 is beginning.
	Variable 55 is beginning.
	Variable 56 is beginning.
	Variable 57 is beginning.
	Variable 58 is beginning.
	Variable 59 is beginning.
	Variable 60 is beginning.
	Variable 61 is beginning.
	Variable 62 is beginning.
	Variable 63 is beginning.
	Variable 64 is beginning.
	Variable 65 is beginning.
	Variable 66 is beginning.
	Variable 67 is beginning.
	Variable 68 is beginning.
	Variable 69 is beginning.
	

### VARVaried - ARDL - Spearman - F-test

In [16]:
rootdir = "../"
dataset = "VARVaried/returns"


association_list = SpearmanMultivariate
associationName = "Spearman"
association_config_list = {"return_type": "p-value",
                           "lags": 10,
                           "selection_rule": "max"}


model_constructor = ARDLModel
modelName = "ARDL"
model_config = {
    "constructor":{
            "lags": 10,
            "order": 10,
            "causal": True,
            "trend": "ct",
            "seasonal": False,
            "period": None,
            "missing": "drop"
        },
        "fit":{
            "cov_type": "HC0"
        }
    }

stoppingName = "f-test"
config = {"significance_threshold": 0.05,
          "method": "f-test",
          "max_features": 10,
          "choose_oracle": True,
          "stop_oracle": False,
          "association": association_list,
          "association.config": association_config_list,
          "model": model_constructor,
          "model.config": model_config}

df_results = run_trials(rootdir,dataset, config,
                        select_random_targets = False, number_selections = 20)

df = pd.DataFrame(df_results)
df["model"]=modelName
df["association"]=associationName
df["stopping_criterion"]=stoppingName
df.to_csv("ARDL_varvaried.csv", mode="a", index=False, header=False)

File data_1.csv has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
	Variable 5 is beginning.
	Variable 6 is beginning.
	Variable 7 is beginning.
	Variable 8 is beginning.
	Variable 9 is beginning.
File data_11.csv has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
	Variable 5 is beginning.
	Variable 6 is beginning.
	Variable 7 is beginning.
	Variable 8 is beginning.
	Variable 9 is beginning.
	Variable 10 is beginning.
	Variable 11 is beginning.
	Variable 12 is beginning.
	Variable 13 is beginning.
	Variable 14 is beginning.
	Variable 15 is beginning.
	Variable 16 is beginning.
	Variable 17 is beginning.
	Variable 18 is beginning.
	Variable 19 is beginning.
	Variable 20 is beginning.
	Variable 21 is beginning.
	Variable 22 is beginning.
	Variable 23 is beginning.
	Variable 24 is beginning.
	Variable 25 is begi

	Variable 216 is beginning.
	Variable 217 is beginning.
	Variable 218 is beginning.
	Variable 219 is beginning.
	Variable 220 is beginning.
	Variable 221 is beginning.
	Variable 222 is beginning.
	Variable 223 is beginning.
	Variable 224 is beginning.
	Variable 225 is beginning.
	Variable 226 is beginning.
	Variable 227 is beginning.
	Variable 228 is beginning.
	Variable 229 is beginning.
	Variable 230 is beginning.
	Variable 231 is beginning.
	Variable 232 is beginning.
	Variable 233 is beginning.
	Variable 234 is beginning.
	Variable 235 is beginning.
	Variable 236 is beginning.
	Variable 237 is beginning.
	Variable 238 is beginning.
	Variable 239 is beginning.
	Variable 240 is beginning.
	Variable 241 is beginning.
	Variable 242 is beginning.
	Variable 243 is beginning.
	Variable 244 is beginning.
	Variable 245 is beginning.
	Variable 246 is beginning.
	Variable 247 is beginning.
	Variable 248 is beginning.
	Variable 249 is beginning.
	Variable 250 is beginning.
	Variable 251 is beg

	Variable 33 is beginning.
	Variable 34 is beginning.
	Variable 35 is beginning.
	Variable 36 is beginning.
	Variable 37 is beginning.
	Variable 38 is beginning.
	Variable 39 is beginning.
	Variable 40 is beginning.
	Variable 41 is beginning.
	Variable 42 is beginning.
	Variable 43 is beginning.
	Variable 44 is beginning.
	Variable 45 is beginning.
	Variable 46 is beginning.
	Variable 47 is beginning.
	Variable 48 is beginning.
	Variable 49 is beginning.
	Variable 50 is beginning.
	Variable 51 is beginning.
	Variable 52 is beginning.
	Variable 53 is beginning.
	Variable 54 is beginning.
	Variable 55 is beginning.
	Variable 56 is beginning.
	Variable 57 is beginning.
	Variable 58 is beginning.
	Variable 59 is beginning.
	Variable 60 is beginning.
	Variable 61 is beginning.
	Variable 62 is beginning.
	Variable 63 is beginning.
	Variable 64 is beginning.
	Variable 65 is beginning.
	Variable 66 is beginning.
	Variable 67 is beginning.
	Variable 68 is beginning.
	Variable 69 is beginning.
	

### VarVaried - ARDL - Pearson - wald-test

In [17]:
rootdir = "../"
dataset = "VARVaried/returns"


association_list = PearsonMultivariate
associationName = "Pearson"
association_config_list = {"return_type": "p-value",
                           "lags": 10,
                           "selection_rule": "max"}


model_constructor = ARDLModel
modelName = "ARDL"
model_config = {
    "constructor":{
            "lags": 10,
            "order": 10,
            "causal": True,
            "trend": "ct",
            "seasonal": False,
            "period": None,
            "missing": "drop"
        },
        "fit":{
            "cov_type": "HC0"
        }
    }

stoppingName = "wald-test"
config = {"significance_threshold": 0.05,
          "method": "wald-test",
          "max_features": 10,
          "choose_oracle": True,
          "stop_oracle": False,
          "association": association_list,
          "association.config": association_config_list,
          "model": model_constructor,
          "model.config": model_config}

df_results = run_trials(rootdir,dataset, config,
                        select_random_targets = False, number_selections = 20)

df = pd.DataFrame(df_results)
df["model"]=modelName
df["association"]=associationName
df["stopping_criterion"]=stoppingName
df.to_csv("ARDL_varvaried.csv", mode="a", index=False, header=False)

File data_1.csv has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
	Variable 5 is beginning.
	Variable 6 is beginning.
	Variable 7 is beginning.
	Variable 8 is beginning.
	Variable 9 is beginning.
File data_11.csv has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
	Variable 5 is beginning.
	Variable 6 is beginning.
	Variable 7 is beginning.
	Variable 8 is beginning.
	Variable 9 is beginning.
	Variable 10 is beginning.
	Variable 11 is beginning.
	Variable 12 is beginning.
	Variable 13 is beginning.
	Variable 14 is beginning.
	Variable 15 is beginning.
	Variable 16 is beginning.
	Variable 17 is beginning.
	Variable 18 is beginning.
	Variable 19 is beginning.
	Variable 20 is beginning.
	Variable 21 is beginning.
	Variable 22 is beginning.
	Variable 23 is beginning.
	Variable 24 is beginning.
	Variable 25 is begi

	Variable 216 is beginning.
	Variable 217 is beginning.
	Variable 218 is beginning.
	Variable 219 is beginning.
	Variable 220 is beginning.
	Variable 221 is beginning.
	Variable 222 is beginning.
	Variable 223 is beginning.
	Variable 224 is beginning.
	Variable 225 is beginning.
	Variable 226 is beginning.
	Variable 227 is beginning.
	Variable 228 is beginning.
	Variable 229 is beginning.
	Variable 230 is beginning.
	Variable 231 is beginning.
	Variable 232 is beginning.
	Variable 233 is beginning.
	Variable 234 is beginning.
	Variable 235 is beginning.
	Variable 236 is beginning.
	Variable 237 is beginning.
	Variable 238 is beginning.
	Variable 239 is beginning.
	Variable 240 is beginning.
	Variable 241 is beginning.
	Variable 242 is beginning.
	Variable 243 is beginning.
	Variable 244 is beginning.
	Variable 245 is beginning.
	Variable 246 is beginning.
	Variable 247 is beginning.
	Variable 248 is beginning.
	Variable 249 is beginning.
	Variable 250 is beginning.
	Variable 251 is beg

	Variable 33 is beginning.
	Variable 34 is beginning.
	Variable 35 is beginning.
	Variable 36 is beginning.
	Variable 37 is beginning.
	Variable 38 is beginning.
	Variable 39 is beginning.
	Variable 40 is beginning.
	Variable 41 is beginning.
	Variable 42 is beginning.
	Variable 43 is beginning.
	Variable 44 is beginning.
	Variable 45 is beginning.
	Variable 46 is beginning.
	Variable 47 is beginning.
	Variable 48 is beginning.
	Variable 49 is beginning.
	Variable 50 is beginning.
	Variable 51 is beginning.
	Variable 52 is beginning.
	Variable 53 is beginning.
	Variable 54 is beginning.
	Variable 55 is beginning.
	Variable 56 is beginning.
	Variable 57 is beginning.
	Variable 58 is beginning.
	Variable 59 is beginning.
	Variable 60 is beginning.
	Variable 61 is beginning.
	Variable 62 is beginning.
	Variable 63 is beginning.
	Variable 64 is beginning.
	Variable 65 is beginning.
	Variable 66 is beginning.
	Variable 67 is beginning.
	Variable 68 is beginning.
	Variable 69 is beginning.
	

### VARVaried - ARDL - Spearman - wald-test

In [18]:
rootdir = "../"
dataset = "VARVaried/returns"


association_list = SpearmanMultivariate
associationName = "Spearman"
association_config_list = {"return_type": "p-value",
                           "lags": 10,
                           "selection_rule": "max"}


model_constructor = ARDLModel
modelName = "ARDL"
model_config = {
    "constructor":{
            "lags": 10,
            "order": 10,
            "causal": True,
            "trend": "ct",
            "seasonal": False,
            "period": None,
            "missing": "drop"
        },
        "fit":{
            "cov_type": "HC0"
        }
    }

stoppingName = "wald-test"
config = {"significance_threshold": 0.05,
          "method": "wald-test",
          "max_features": 10,
          "choose_oracle": True,
          "stop_oracle": False,
          "association": association_list,
          "association.config": association_config_list,
          "model": model_constructor,
          "model.config": model_config}

df_results = run_trials(rootdir,dataset, config,
                        select_random_targets = False, number_selections = 20)

df = pd.DataFrame(df_results)
df["model"]=modelName
df["association"]=associationName
df["stopping_criterion"]=stoppingName
df.to_csv("ARDL_varvaried.csv", mode="a", index=False, header=False)

File data_1.csv has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
	Variable 5 is beginning.
	Variable 6 is beginning.
	Variable 7 is beginning.
	Variable 8 is beginning.
	Variable 9 is beginning.
File data_11.csv has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
	Variable 5 is beginning.
	Variable 6 is beginning.
	Variable 7 is beginning.
	Variable 8 is beginning.
	Variable 9 is beginning.
	Variable 10 is beginning.
	Variable 11 is beginning.
	Variable 12 is beginning.
	Variable 13 is beginning.
	Variable 14 is beginning.
	Variable 15 is beginning.
	Variable 16 is beginning.
	Variable 17 is beginning.
	Variable 18 is beginning.
	Variable 19 is beginning.
	Variable 20 is beginning.
	Variable 21 is beginning.
	Variable 22 is beginning.
	Variable 23 is beginning.
	Variable 24 is beginning.
	Variable 25 is begi

	Variable 216 is beginning.
	Variable 217 is beginning.
	Variable 218 is beginning.
	Variable 219 is beginning.
	Variable 220 is beginning.
	Variable 221 is beginning.
	Variable 222 is beginning.
	Variable 223 is beginning.
	Variable 224 is beginning.
	Variable 225 is beginning.
	Variable 226 is beginning.
	Variable 227 is beginning.
	Variable 228 is beginning.
	Variable 229 is beginning.
	Variable 230 is beginning.
	Variable 231 is beginning.
	Variable 232 is beginning.
	Variable 233 is beginning.
	Variable 234 is beginning.
	Variable 235 is beginning.
	Variable 236 is beginning.
	Variable 237 is beginning.
	Variable 238 is beginning.
	Variable 239 is beginning.
	Variable 240 is beginning.
	Variable 241 is beginning.
	Variable 242 is beginning.
	Variable 243 is beginning.
	Variable 244 is beginning.
	Variable 245 is beginning.
	Variable 246 is beginning.
	Variable 247 is beginning.
	Variable 248 is beginning.
	Variable 249 is beginning.
	Variable 250 is beginning.
	Variable 251 is beg

	Variable 33 is beginning.
	Variable 34 is beginning.
	Variable 35 is beginning.
	Variable 36 is beginning.
	Variable 37 is beginning.
	Variable 38 is beginning.
	Variable 39 is beginning.
	Variable 40 is beginning.
	Variable 41 is beginning.
	Variable 42 is beginning.
	Variable 43 is beginning.
	Variable 44 is beginning.
	Variable 45 is beginning.
	Variable 46 is beginning.
	Variable 47 is beginning.
	Variable 48 is beginning.
	Variable 49 is beginning.
	Variable 50 is beginning.
	Variable 51 is beginning.
	Variable 52 is beginning.
	Variable 53 is beginning.
	Variable 54 is beginning.
	Variable 55 is beginning.
	Variable 56 is beginning.
	Variable 57 is beginning.
	Variable 58 is beginning.
	Variable 59 is beginning.
	Variable 60 is beginning.
	Variable 61 is beginning.
	Variable 62 is beginning.
	Variable 63 is beginning.
	Variable 64 is beginning.
	Variable 65 is beginning.
	Variable 66 is beginning.
	Variable 67 is beginning.
	Variable 68 is beginning.
	Variable 69 is beginning.
	

### VARVaried - ARDL - Pearson - lr-test

In [19]:
rootdir = "../"
dataset = "VARVaried/returns"


association_list = PearsonMultivariate
associationName = "Pearson"
association_config_list = {"return_type": "p-value",
                           "lags": 10,
                           "selection_rule": "max"}


model_constructor = ARDLModel
modelName = "ARDL"
model_config = {
    "constructor":{
            "lags": 10,
            "order": 10,
            "causal": True,
            "trend": "ct",
            "seasonal": False,
            "period": None,
            "missing": "drop"
        },
        "fit":{
            "cov_type": "HC0"
        }
    }

stoppingName = "lr-test"
config = {"significance_threshold": 0.05,
          "method": "lr-test",
          "max_features": 10,
          "choose_oracle": True,
          "stop_oracle": False,
          "association": association_list,
          "association.config": association_config_list,
          "model": model_constructor,
          "model.config": model_config}

df_results = run_trials(rootdir,dataset, config,
                        select_random_targets = False, number_selections = 20)

df = pd.DataFrame(df_results)
df["model"]=modelName
df["association"]=associationName
df["stopping_criterion"]=stoppingName
df.to_csv("ARDL_varvaried.csv", mode="a", index=False, header=False)

File data_1.csv has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
	Variable 5 is beginning.
	Variable 6 is beginning.
	Variable 7 is beginning.
	Variable 8 is beginning.
	Variable 9 is beginning.
File data_11.csv has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
	Variable 5 is beginning.
	Variable 6 is beginning.
	Variable 7 is beginning.
	Variable 8 is beginning.
	Variable 9 is beginning.
	Variable 10 is beginning.
	Variable 11 is beginning.
	Variable 12 is beginning.
	Variable 13 is beginning.
	Variable 14 is beginning.
	Variable 15 is beginning.
	Variable 16 is beginning.
	Variable 17 is beginning.
	Variable 18 is beginning.
	Variable 19 is beginning.
	Variable 20 is beginning.
	Variable 21 is beginning.
	Variable 22 is beginning.
	Variable 23 is beginning.
	Variable 24 is beginning.
	Variable 25 is begi

	Variable 216 is beginning.
	Variable 217 is beginning.
	Variable 218 is beginning.
	Variable 219 is beginning.
	Variable 220 is beginning.
	Variable 221 is beginning.
	Variable 222 is beginning.
	Variable 223 is beginning.
	Variable 224 is beginning.
	Variable 225 is beginning.
	Variable 226 is beginning.
	Variable 227 is beginning.
	Variable 228 is beginning.
	Variable 229 is beginning.
	Variable 230 is beginning.
	Variable 231 is beginning.
	Variable 232 is beginning.
	Variable 233 is beginning.
	Variable 234 is beginning.
	Variable 235 is beginning.
	Variable 236 is beginning.
	Variable 237 is beginning.
	Variable 238 is beginning.
	Variable 239 is beginning.
	Variable 240 is beginning.
	Variable 241 is beginning.
	Variable 242 is beginning.
	Variable 243 is beginning.
	Variable 244 is beginning.
	Variable 245 is beginning.
	Variable 246 is beginning.
	Variable 247 is beginning.
	Variable 248 is beginning.
	Variable 249 is beginning.
	Variable 250 is beginning.
	Variable 251 is beg

	Variable 33 is beginning.
	Variable 34 is beginning.
	Variable 35 is beginning.
	Variable 36 is beginning.
	Variable 37 is beginning.
	Variable 38 is beginning.
	Variable 39 is beginning.
	Variable 40 is beginning.
	Variable 41 is beginning.
	Variable 42 is beginning.
	Variable 43 is beginning.
	Variable 44 is beginning.
	Variable 45 is beginning.
	Variable 46 is beginning.
	Variable 47 is beginning.
	Variable 48 is beginning.
	Variable 49 is beginning.
	Variable 50 is beginning.
	Variable 51 is beginning.
	Variable 52 is beginning.
	Variable 53 is beginning.
	Variable 54 is beginning.
	Variable 55 is beginning.
	Variable 56 is beginning.
	Variable 57 is beginning.
	Variable 58 is beginning.
	Variable 59 is beginning.
	Variable 60 is beginning.
	Variable 61 is beginning.
	Variable 62 is beginning.
	Variable 63 is beginning.
	Variable 64 is beginning.
	Variable 65 is beginning.
	Variable 66 is beginning.
	Variable 67 is beginning.
	Variable 68 is beginning.
	Variable 69 is beginning.
	

### VARVaried - ARDL - Spearman - lr-test

In [20]:
rootdir = "../"
dataset = "VARVaried/returns"


association_list = SpearmanMultivariate
associationName = "Spearman"
association_config_list = {"return_type": "p-value",
                           "lags": 10,
                           "selection_rule": "max"}


model_constructor = ARDLModel
modelName = "ARDL"
model_config = {
    "constructor":{
            "lags": 10,
            "order": 10,
            "causal": True,
            "trend": "ct",
            "seasonal": False,
            "period": None,
            "missing": "drop"
        },
        "fit":{
            "cov_type": "HC0"
        }
    }

stoppingName = "lr-test"
config = {"significance_threshold": 0.05,
          "method": "lr-test",
          "max_features": 10,
          "choose_oracle": True,
          "stop_oracle": False,
          "association": association_list,
          "association.config": association_config_list,
          "model": model_constructor,
          "model.config": model_config}

df_results = run_trials(rootdir,dataset, config,
                        select_random_targets = False, number_selections = 20)

df = pd.DataFrame(df_results)
df["model"]=modelName
df["association"]=associationName
df["stopping_criterion"]=stoppingName
df.to_csv("ARDL_varvaried.csv", mode="a", index=False, header=False)

File data_1.csv has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
	Variable 5 is beginning.
	Variable 6 is beginning.
	Variable 7 is beginning.
	Variable 8 is beginning.
	Variable 9 is beginning.
File data_11.csv has begun.
	Variable 0 is beginning.
	Variable 1 is beginning.
	Variable 2 is beginning.
	Variable 3 is beginning.
	Variable 4 is beginning.
	Variable 5 is beginning.
	Variable 6 is beginning.
	Variable 7 is beginning.
	Variable 8 is beginning.
	Variable 9 is beginning.
	Variable 10 is beginning.
	Variable 11 is beginning.
	Variable 12 is beginning.
	Variable 13 is beginning.
	Variable 14 is beginning.
	Variable 15 is beginning.
	Variable 16 is beginning.
	Variable 17 is beginning.
	Variable 18 is beginning.
	Variable 19 is beginning.
	Variable 20 is beginning.
	Variable 21 is beginning.
	Variable 22 is beginning.
	Variable 23 is beginning.
	Variable 24 is beginning.
	Variable 25 is begi

	Variable 216 is beginning.
	Variable 217 is beginning.
	Variable 218 is beginning.
	Variable 219 is beginning.
	Variable 220 is beginning.
	Variable 221 is beginning.
	Variable 222 is beginning.
	Variable 223 is beginning.
	Variable 224 is beginning.
	Variable 225 is beginning.
	Variable 226 is beginning.
	Variable 227 is beginning.
	Variable 228 is beginning.
	Variable 229 is beginning.
	Variable 230 is beginning.
	Variable 231 is beginning.
	Variable 232 is beginning.
	Variable 233 is beginning.
	Variable 234 is beginning.
	Variable 235 is beginning.
	Variable 236 is beginning.
	Variable 237 is beginning.
	Variable 238 is beginning.
	Variable 239 is beginning.
	Variable 240 is beginning.
	Variable 241 is beginning.
	Variable 242 is beginning.
	Variable 243 is beginning.
	Variable 244 is beginning.
	Variable 245 is beginning.
	Variable 246 is beginning.
	Variable 247 is beginning.
	Variable 248 is beginning.
	Variable 249 is beginning.
	Variable 250 is beginning.
	Variable 251 is beg

	Variable 33 is beginning.
	Variable 34 is beginning.
	Variable 35 is beginning.
	Variable 36 is beginning.
	Variable 37 is beginning.
	Variable 38 is beginning.
	Variable 39 is beginning.
	Variable 40 is beginning.
	Variable 41 is beginning.
	Variable 42 is beginning.
	Variable 43 is beginning.
	Variable 44 is beginning.
	Variable 45 is beginning.
	Variable 46 is beginning.
	Variable 47 is beginning.
	Variable 48 is beginning.
	Variable 49 is beginning.
	Variable 50 is beginning.
	Variable 51 is beginning.
	Variable 52 is beginning.
	Variable 53 is beginning.
	Variable 54 is beginning.
	Variable 55 is beginning.
	Variable 56 is beginning.
	Variable 57 is beginning.
	Variable 58 is beginning.
	Variable 59 is beginning.
	Variable 60 is beginning.
	Variable 61 is beginning.
	Variable 62 is beginning.
	Variable 63 is beginning.
	Variable 64 is beginning.
	Variable 65 is beginning.
	Variable 66 is beginning.
	Variable 67 is beginning.
	Variable 68 is beginning.
	Variable 69 is beginning.
	

In [40]:
for filename in os.listdir(rootdir + "data/" + dataset + "/"):
        if not os.path.isfile(rootdir + "data/" + dataset + "/" + filename):
            continue
        print("File",filename,"has begun.")

        df, _, causes = data_opener.open_dataset_and_ground_truth(dataset, filename, rootdir=rootdir)
        
        df_train, df_test = separate_train_test(df)

        targets = np.random.choice(df.columns, size=number_selections) if False else df.columns

        for target in targets:
            if len(causes[target]) == 0:
                continue
            else:
                break
        break

File data_0.csv has begun.


In [41]:
causes_autoregressive = causes[target] if target in causes[target] else causes[target]+[target]
model = config["model"](config["model.config"], target)
# potentially remove the target from the predictors with lag=0, but for now let this be.
model.fit(df_train[causes_autoregressive])

In [42]:
res = model.predict(df_test[causes_autoregressive])

In [43]:
res.model.predict(res._params)

AttributeError: 'ARDL' object has no attribute 'model'

In [ ]:
#params, exog, exog_oos, start, end, num_oos
res.model._prepare_prediction(res._params, exog=None, exog_oos=None, start=None, end=None)

In [ ]:
res.model._index

In [ ]:
res.model._get_prediction_index(0, 4000)

In [ ]:
res.model.predict(res._params)

In [ ]:
res._params

In [ ]:
res.params.index

In [ ]:
model.results.params

In [ ]:
model = config["model"](config["model.config"], target)
model.fit(df_train[[target]])
res = model.results.apply(endog = df_test[target])

In [ ]:
res.resid


In [ ]:
print(res.model.data.__dict__)

In [ ]:
plt.plot(res.model.predict(res._params, start=0,end=1199, dynamic=False))
plt.subplots()
plt.plot(res.model.data.orig_endog)

In [ ]:
pd.Series.sum(res.model.predict(res._params, start=0,end=1199, dynamic=False) - res.model.data.orig_endog)

In [ ]:
causes_autoregressive = causes[target] if target in causes[target] else causes[target]+[target]
model = config["model"](config["model.config"], target)
# potentially remove the target from the predictors with lag=0, but for now let this be.
model.fit(df_train[causes_autoregressive])

In [ ]:
res = model.predict(df_test[causes_autoregressive])

In [ ]:
res.model.data.__dict__

In [ ]:
model.results.params

In [ ]:
model2 = ARDLModel(config["model.config"], target)
model2.fit(df_test[causes_autoregressive])

In [ ]:
model2.results.model.predict(model.results._params, start=0, end=len(df_test)-1)

In [ ]:
print(model2.results.llf)
model2.results._params=model.results._params
print(model2.results.llf)

In [ ]:
constraint_matrix = []
for i, param_name in enumerate(model.results.params.index):
    if "B" in param_name:
        new_constraint = np.zeros((len(model.results.params.index),))
        new_constraint[i]=1
        constraint_matrix.append(new_constraint)
    
r_matrix = np.array(constraint_matrix)
model.results.wald_test(r_matrix, use_f=False, scalar=True)